In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl
from tqdm.auto import tqdm
from src.settings import *
from src.tools.osmnx_utils import get_place_dir_name
from src.tools.vis_utils import visualize_kepler, save_config
tqdm.pandas()

In [3]:
cities = pd.read_csv(RAW_DATA_DIR / "cities.csv")
cities = cities[(cities.country == "Poland") & (cities.kacper)]
cities

,city,country,kacper,szymon,piotr,kamil,regions
63,Białystok,Poland,True,False,False,False,NaN
64,Bydgoszcz,Poland,True,False,True,False,NaN
65,Częstochowa,Poland,True,False,False,False,NaN
69,Kraków,Poland,True,True,True,False,NaN
70,Lublin,Poland,True,False,True,False,NaN
71,Łódź,Poland,True,True,False,False,NaN
72,Poznań,Poland,True,True,True,True,NaN
75,Szczecin,Poland,True,True,True,False,NaN
77,Trójmiasto,Poland,True,False,True,False,"['Gdańsk','Gdynia','Sopot']"
78,Warszawa,Poland,True,True,True,True,NaN


In [4]:
network_type = "drive"

pbar = tqdm(cities.itertuples(), total=cities.shape[0])
places = []
for row in pbar:
    place_name = f"{row.city},{row.country}"
    place_dir_name = get_place_dir_name(place_name)
    pbar.set_description(place_name)

    try:
        place_df = gpd.read_file(GENERATED_DATA_DIR / place_dir_name / f"graph_{network_type}.gpkg", layer="place")
        place_df["city"] = row.city
        place_df["country"] = row.country
        places.append(place_df)
    except:
        print("\nFailed", place_name)
    

df_places = pd.concat(places, ignore_index=True)

Zielona Góra,Poland: 100%|██████████| 12/12 [00:00<00:00, 25.43it/s]


In [5]:
config_name = "cities"
m = visualize_kepler(df_places, config_name=config_name)
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'iuc031q', 'type': …

In [6]:
save_config(m, config_name=config_name)

WindowsPath('D:/Projekty/Programowanie/Studia/Semestr10/gis/osm-road-infrastructure/assets/keplergl/config/cities.json')